#  This notebook is for the Capstone Project of the Coursera course Applied Data Science
** First the postcodes of Toronto will be scraped from a wikipeda page. This data will be stored and processed into a Data Frame.**

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import geocoder
import folium
from geopy.geocoders import Nominatim

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

**Get the HTML code from the wikipedia website with the postcodes**

In [6]:
#get the html code from the respective website
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
#store the html file in the soup variable with the lxml parser
soup = BeautifulSoup(page, 'lxml')


**Initialize some empty lists**

In [7]:
# initialize some variables
columns_headers = []
values_column1 = []
values_column2 = []
values_column3 = []
filter_DF =[]
lat_lng_coords = None
latitude = []
longitude = []

** Scrape the webpage. We will get the name for the columns as well as the data for each row from the HTML code**

In [8]:
# get the relevent div from the whole soup
source = soup.find('div', class_='mw-parser-output')

In [9]:
# search for all entries in the first row of the table (the headings)
for tr in source.find_all('tr')[0].find_all('th'):
    columns_headers.append(tr.text)

In [10]:
# because the last heading has a \n at the end, we have the cut it away
columns_headers[2] = columns_headers[2][0:-1]

In [11]:
# loop through the table to get all the entries for the columns
for td in source.find_all('td')[::3]:
    values_column1.append(td.text)
for td in source.find_all('td')[1::3]:
    values_column2.append(td.text)    
for td in source.find_all('td')[2::3]:
    values_column3.append(td.text)    
    

In [12]:
# because there is some extra code that is not part of the table we have to get rid of the irrelevant entries
relevant_number_rows = values_column1.index('M9Z')
values_column1 = values_column1[0:relevant_number_rows+1]
values_column2 = values_column2[0:relevant_number_rows+1]
values_column3 = values_column3[0:relevant_number_rows+1]

#again in column 3 there is a /n at the end that has be truncated
for i,value in enumerate(values_column3):
    values_column3[i-1] = values_column3[i-1][0:-1]

**We will now create the data frame with the data we have obtained**
**Then we will wrangle the data, so we can better process it in a later stage**

In [13]:
# create a data frame
df_values = np.array([values_column1,values_column2,values_column3]).transpose()
Neighborhoods_CN = pd.DataFrame(data = df_values,columns = columns_headers)


In [14]:
#drop all the rows with 'Borough' = Not assinged
Neighborhoods_CN['Borough'].replace(to_replace='Not assigned',value = np.nan,inplace=True)
Neighborhoods_CN.dropna(inplace=True)
Neighborhoods_CN.reset_index(drop = True,inplace = True)


In [15]:
unique_postcodes = Neighborhoods_CN['Postcode'].unique().tolist()


In [16]:
# getting a list with the boolean values for unique postcodes
length = len(Neighborhoods_CN['Postcode'])

for i in range(0,length):
    filter_DF.append(Neighborhoods_CN['Postcode'][i] in unique_postcodes)
    if Neighborhoods_CN['Postcode'][i] in unique_postcodes:     
        unique_postcodes.pop(unique_postcodes.index(Neighborhoods_CN['Postcode'][i]))
        
filter_DF_reversed = list(~np.array(filter_DF))

In [17]:
# creating two new data frames. The first one with single postcodes.
# The second one with the rest of the intial data frame
Neighborhoods_CN_filtered1 = Neighborhoods_CN[filter_DF]
Neighborhoods_CN_filtered2 = Neighborhoods_CN[filter_DF_reversed]

for val in Neighborhoods_CN_filtered2['Postcode'].values.tolist():
    for i in Neighborhoods_CN_filtered2[Neighborhoods_CN_filtered2['Postcode']==val].index:
        row = Neighborhoods_CN_filtered1[Neighborhoods_CN_filtered1['Postcode']==val].index[0]
        val_to_add =  Neighborhoods_CN_filtered2.loc[i,'Neighbourhood']
        Neighborhoods_CN_filtered1.loc[row,'Neighbourhood'] = Neighborhoods_CN_filtered1.loc[row,'Neighbourhood'] + ', ' + val_to_add

C:\Users\netze\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\netze\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [18]:
# now we append the Neighbourhood of the second data frame to the first one for the respective postcodes 
for i in Neighborhoods_CN_filtered1[Neighborhoods_CN_filtered1['Neighbourhood'] == 'Not assigned']['Borough'].index:
    val = Neighborhoods_CN_filtered1.loc[i,'Borough']
    Neighborhoods_CN_filtered1.replace(to_replace='Not assigned', value=val, inplace =True)

C:\Users\netze\Anaconda3\lib\site-packages\pandas\core\frame.py:3795: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


In [19]:
Neighborhoods_CN_filtered1.shape


(103, 3)

**Get the coordinates to the differen postcodes. Then sort both data frames (to one with the coordinates and the one where the coordinates will be added) by the postcode, so to coordinates match. Then add the coordinates to the data frame**

In [30]:
#tried to get the coordinated with this for and while loop, but it ran endlessly

#for p_code in Neighborhoods_CN_filtered1['Postcode'].values.tolist():
#    while(lat_lng_coords is None):
#        g = geocoder.google('{}, Toronto, Ontario'.format(p_code))
#        lat_lng_coords = g.latlng
#    latitude.append(lat_lng_coords[0])
#    longitude.append(lat_lng_coords[1])

# read the csv file into a data frame
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
#sort by Postcode
coordinates.sort_values(by=['Postal Code'], inplace=True)
# save the latitude and longitude as lists
latitude = coordinates['Latitude'].values.tolist()
longitude = coordinates['Longitude'].values.tolist()

In [21]:
# also sort the main data frame by postcode and reset the index afterwards
Neighborhoods_CN_filtered1.sort_values(by = ['Postcode'], inplace = True)
Neighborhoods_CN_filtered1.reset_index(drop = True, inplace = True)
#add the latitude and longitude
Neighborhoods_CN_filtered1['Latitude'] = latitude
Neighborhoods_CN_filtered1['Longitude'] = longitude

C:\Users\netze\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\netze\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\netze\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


** Visualize all the different neighborhoods in Toronto**

In [29]:

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude[0], longitude[0]], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(Neighborhoods_CN_filtered1['Latitude'], Neighborhoods_CN_filtered1['Longitude'], Neighborhoods_CN_filtered1['Borough'], Neighborhoods_CN_filtered1['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7
            ).add_to(map_toronto)  
    
map_toronto

In [26]:
#Neighborhoods_CN_filtered1